In [1]:
from imgaug import augmenters as iaa
from keras import applications, optimizers
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
import cv2
import keras
import os, csv
import shutil
import imgaug as ia
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
# dimensions the images.
img_width, img_height = 512, 384

train_data_dir = '../dataset/train'
test_data_dir = '../dataset/test'
validation_data_dir = '../dataset/validation'
original_data_dir = '../dataset-resized'

batch_size = 16
epochs = 20

In [3]:
def buildModel():
    # build the VGG16 network
    base_model = applications.VGG16(weights='imagenet', 
                                    include_top=False,
                                    input_tensor=Input(shape=(img_width, img_height, 3)))

    for layer in base_model.layers:
        layer.trainable = False
        
    top_model = base_model.output
    top_model = Flatten(name="Flatten")(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(256, activation='relu')(top_model)
    top_model = Dense(6, activation='softmax')(top_model)
    
    model = Model(inputs=base_model.input, outputs=top_model)

#     model.summary()
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

In [4]:
def createStratifiedData():
    # create dataframe with the images filenames
    dataset_files = open('./dataset.csv', 'w+')
    writer = csv.writer(dataset_files)

    writer.writerow(['image','class'])
    for path, dirs, files in os.walk(original_data_dir):
        for filename in files:
            if( filename != '.DS_Store'):
                writer.writerow([filename, os.path.basename(path)])

    # dataframe containing the filenames of the images and the classes
    df = pd.read_csv('./dataset.csv')
    df_y = df['class']
    df_x = df['image']

    skf = StratifiedShuffleSplit(n_splits = 1, test_size=0.2)

    for train_index, test_index in skf.split(df_x, df_y):
        x_train, x_test = df_x[train_index], df_x[test_index]
        y_train, y_test = df_y[train_index], df_y[test_index]

        train = pd.concat([x_train, y_train], axis=1)
        test = pd.concat([x_test, y_test], axis = 1)
        # take 20% of the training data from this fold for validation during training
        validation = test.sample(frac = 0.5)

        # make sure validation data does not include training data
        train = train[~train['image'].isin(list(validation['image']))]

        # copy the images according to the fold
        copy_images(train, 'train')
        copy_images(validation, 'validation')
        copy_images(test, 'test')

In [5]:
# used to copy files according to each fold
def copy_images(dataframe, directory):
    destination_directory = './dataset/{}'.format(directory)
    print('copying {} files to {}...'.format(directory, destination_directory))

    # remove all files from previous fold
    if os.path.exists(destination_directory):
        shutil.rmtree(destination_directory)

    # create folder for files from this fold
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    # create subfolders for each class
    for image_class in set(list(dataframe['class'])):
        if not os.path.exists(destination_directory + '/' + image_class):
            os.makedirs(destination_directory + '/' + image_class)

    # copy files for this fold from a directory holding all the files
    for i, row in dataframe.iterrows():
        try:
            # this is the path to all of your images kept together in a separate folder
            path_from = '{}/{}/{}'.format(original_data_dir, row['class'],row['image'])
            path_to = "{}/{}".format(destination_directory, row['class'])
            
            # move from folder keeping all files to training, test, or validation folder (the "directory" argument)
            shutil.copy(path_from, path_to)
        except Exception as e:
            print("Error when copying {}: {}".format(row['image'], str(e)))

In [6]:
def augmentData():        
    # create numpy array with images paths
    img_paths = np.array([])
    for path, dirs, files in os.walk(train_data_dir):
        for filename in files:
            if(filename != '.DS_Store'):
                path_name = '{}/{}/{}'.format(train_data_dir,os.path.basename(path),filename)
                img_paths = np.append(img_paths, path_name)   
                
    # convert images to numpy arrays
    paths_number, = img_paths.shape
    
    images = np.zeros(shape=(paths_number,img_height,img_width,3),dtype='uint8')
    for idx, img_path in enumerate(img_paths):
#         print('Creating array from ', img_path)
        img = cv2.imread(img_path, 1)
        im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images[idx] = im_rgb
        
    # aug configurations
    seq = iaa.Sequential([
        iaa.Affine(scale=(0.8, 1.5)), # Scale images to a value between 80% and 150%
        iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}), # Translate images by -10 to +10% on x- and y-axis independently
        iaa.Affine(rotate=(-25, 25)), # Rotate images by -45 to 45 degrees
        iaa.GaussianBlur(sigma=(0.0, 2.0)), # Blur each image with a gaussian kernel with a sigma of 2.0:
        iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)), # Add gaussian noise to an image, sampled once per pixel from a normal distribution N(0, s), where s is sampled per image and varies between 0 and 0.05*255
        iaa.Fliplr(0.5), # Flip 50% of all images horizontally
        iaa.Flipud(0.5), # Flip 50% of all images vertically
        iaa.CropAndPad(
                    percent=(-0.1, 0.1),
                    pad_mode=["constant", "edge"],
                    pad_cval=(0, 128)) # Crop or pad each side by up to 10 percent relative to its original size
    ], random_order=True)

    # create augmented images
    images_aug = seq.augment_images(images)
    
    # save augmented images
    for i in range(len(img_paths)):
        img_dest = '{}_AUG.jpg'.format(img_paths[i][:-4])  
        Image.fromarray(images_aug[i]).save(img_dest)
#         print('Saving ',img_dest)

In [7]:
def generateData(batch_size):
    train_datagen = ImageDataGenerator()

    validation_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')
    
    return [train_generator,validation_generator]

In [8]:
def fineTuneModel(model, train_generator, validation_generator, epochs, batch_size):
    training_samples = 0
    validation_samples = 0
    
    for path, dirs, files in os.walk(train_data_dir):
        for filename in files:
            training_data += 1 

    for path, dirs, files in os.walk(validation_data_dir):
        for filename in files:
            validation_data += 1 
        
    model.fit_generator(
        train_generator,
        steps_per_epoch=training_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples // batch_size)
    
    return model

In [9]:
def getMetrics(model, batch_size):
    
    test_datagen = ImageDataGenerator(rescale=1. /255)

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width,img_height),
        shuffle=False,
        batch_size=1,
        class_mode='categorical')
    
    predicted_results = model.predict_generator(test_generator, 
                                                steps = nb_test_samples)
    predicted_results = np.argmax(predicted_results, axis=1)
    targets = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
    
    # confusion matrix
    print('CONFUSION MATRIX:')
    print(confusion_matrix(test_generator.classes, predicted_results))
    
    # classification report
    print('CLASSIFICATION REPORT:')
    print(classification_report(test_generator.classes, predicted_results, target_names=targets))

In [10]:
def runExperiments():
    batch_sizes = [8,16,32]
    epochs = [10,20,30]
    config = 1
    
    for batch_size in batch_sizes:
        for epoch in epochs:
            
            print("*************** Test ", config, " Batch size: ", batch_size, " Epochs: ", epoch, "***************")           
            keras.backend.clear_session()
            
            model = buildModel()
            train_generator,test_generator = generateData(batch_size)
            trained_model = fineTuneModel(model,train_generator, test_generator, epoch, batch_size)
            metrics = getMetrics(trained_model, batch_size)
            
            config += 1

In [11]:
runExperiments()

W0831 19:33:11.329114 139719852369664 deprecation_wrapper.py:119] From /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0831 19:33:11.331288 139719852369664 deprecation_wrapper.py:119] From /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0831 19:33:11.348122 139719852369664 deprecation_wrapper.py:119] From /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0831 19:33:11.348794 139719852369664 deprecation_wrapper.py:119] From /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/keras/backend/tensorflow_backend.p

*************** Test  1  Batch size:  8  Epochs:  10 ***************


ResourceExhaustedError: OOM when allocating tensor with shape[128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node block2_conv2/bias/Assign (defined at /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:402) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Errors may have originated from an input operation.
Input Source operations connected to node block2_conv2/bias/Assign:
 block2_conv2/Const (defined at /home/martin/anaconda3/envs/martin/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:430)

Original stack trace for 'block2_conv2/bias/Assign':
  File "/home/martin/anaconda3/envs/martin/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/martin/anaconda3/envs/martin/lib/python3.6/runpy.py", line 85, in _run_code
    exec(cod